In [1]:
!pip install langchain_google_genai -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY_1')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key = userdata.get('GOOGLE_API_KEY_1')
)

In [4]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)

        except Exception as e:
            return f"Error: {e}"

In [7]:
from langchain_core.tools import tool
import math

@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

@tool
def square_root(num: float) -> float:
    """
    Calculate the square root of a number.
    Input: A number as a float (e.g., 9).
    Output: Square root of the number as a float.
    """
    return math.sqrt(num)

@tool
def trignometry(num: float, operation: str) -> float:
  """
  Perform trigonometric operations on a number.
  Input: A number as a float (e.g., 30) and an operation as a string (e.g., "sin").
  Output: Result of the trigonometric operation as a float.
  """
  if operation == "sin":
    return math.sin(num)
  elif operation == "csc":
    return 1/math.sin(num)
  elif operation == "cos":
    return math.cos(num)
  elif operation == "sec":
    return 1/math.cos(num)
  elif operation == "tan":
    return math.tan(num)
  elif operation == "cot":
    return 1/math.tan(num)
  else:
    return "Invalid operation"

@tool
def factorial(num: int) -> int:
    """
    Calculate the factorial of a number.
    Input: A non-negative integer (e.g., 5).
    Output: Factorial of the number as an integer.
    """
    return math.factorial(num)

@tool
def logarithm(num: float, base: int) -> float:
    """
    Calculate the logarithm of a number with a specified base.
    Input: A number as a float (e.g., 100) and a base as an integer (e.g., 10).
    Output: Logarithm of the number with the specified base as a float.
    """
    return math.log(num, base)

tools = [calculator, square_root, trignometry, factorial, logarithm]

In [8]:
llm_with_tools = llm.bind_tools(tools)

In [9]:
from langchain_core.messages import HumanMessage, AIMessage
query = input()
messages = [HumanMessage(query)]
display(messages)

What is sin 90


[HumanMessage(content='What is sin 90', additional_kwargs={}, response_metadata={})]

In [10]:
ai_msg_tools = llm_with_tools.invoke(messages)
messages.append(ai_msg_tools)

In [11]:
for tool_call in ai_msg_tools.tool_calls:
    selected_tool = {
        "calculator": calculator,
        "square_root": square_root,
        "trignometry": trignometry,
        "factorial": factorial,
        "logarithm": logarithm
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    messages.append(tool_msg)

ToolMessage(content='0.8939966636005579', name='trignometry', tool_call_id='53ba0414-6d5c-49b1-936e-87fa1d62a976')

In [12]:
response = llm_with_tools.invoke(messages)

In [13]:
from IPython.display import Markdown

Markdown(response.content)

The sine of 90 is approximately 0.894.